<a href="https://colab.research.google.com/github/ykitaguchi77/manipulate_CSV/blob/master/Copy_extracted_images_from_csv_Local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **Copy extracted image from csv file**

• CSVファイルから患者のIDを抽出します。

• 抽出した情報に基づいて、指定されたディレクトリから画像ファイルを検索します。

• 見つかった画像ファイルを新しく作成したフォルダにコピーします。


In [ ]:
# 必要なライブラリをインポート
import os
import shutil
import csv
import tqdm

# CSVファイルから情報を抽出
def extract_csv_info(csv_path):
    with open(csv_path, 'r', encoding='utf-8') as csv_file:
        # CSVの3列目（インデックス2）の値を抽出し、ヘッダー行を除外
        return [row[2] for row in csv.reader(csv_file) if len(row) >= 3][1:]

# 指定されたディレクトリから画像ファイルのパスを見つける関数
def find_image_paths(img_dir, extracted_numbers):
    image_paths = {}
    for root, _, files in os.walk(img_dir):
        for file in tqdm.tqdm(files, desc="ファイル処理中"):
            # 画像ファイルの拡張子をチェック
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                file_name, _ = os.path.splitext(file)
                file_parts = file_name.split('-')
                file_number = file_parts[0]
                # 抽出された番号と一致する画像を見つける
                if file_number in extracted_numbers:
                    current_path = os.path.join(root, file)
                    # 同じ番号の画像が複数ある場合、番号が小さい方を選択
                    if file_number not in image_paths or (len(file_parts) > 1 and int(file_parts[1]) < int(image_paths[file_number].split('-')[1])):
                        image_paths[file_number] = current_path
    return list(image_paths.values())

# 画像をコピーする関数
def copy_images(image_paths, destination_folder):
    for image_path in tqdm.tqdm(image_paths, desc="画像コピー中"):
        shutil.copy(image_path, os.path.join(destination_folder, os.path.basename(image_path)))

# メイン関数
def main():
    # 各種パスを設定
    img_dir = "F:\眼位写真NEW"
    extract_csv_path = r"C:\Users\ykita\OneDrive\デスクトップ\test_images\Disease_extracted_眼瞼下垂.csv"
    copy_dir = r"C:\Users\ykita\OneDrive\デスクトップ\test_images"

        # 新しいフォルダ名を生成し、作成
    csv_basename = os.path.splitext(os.path.basename(extract_csv_path).replace("Disease_extracted_", ""))[0]
    new_folder_path = os.path.join(copy_dir, csv_basename)
    print(f"作成しようとしているフォルダのパス: {new_folder_path}")

    try:
        os.makedirs(new_folder_path, exist_ok=True)
        print(f"フォルダが正常に作成されました: {new_folder_path}")
    except Exception as e:
        print(f"フォルダの作成中にエラーが発生しました: {e}")
        return

    if not os.path.exists(new_folder_path):
        print(f"エラー: フォルダ {new_folder_path} が作成されませんでした。")
        return

    # CSVから情報を抽出し、該当する画像を見つけてコピー
    extracted_numbers = extract_csv_info(extract_csv_path)
    image_paths = find_image_paths(img_dir, extracted_numbers)
    copy_images(image_paths, new_folder_path)

    # 処理完了メッセージを表示
    print(f"コピーが完了しました。{len(image_paths)}個の画像が{new_folder_path}にコピーされました。")

# スクリプトが直接実行された場合にメイン関数を呼び出す
if __name__ == "__main__":
    main()


###一括で行うバージョン

In [ ]:
# 必要なライブラリをインポート
import os
import shutil
import csv
import tqdm
import glob

# CSVファイルから情報を抽出
def extract_csv_info(csv_path):
    with open(csv_path, 'r', encoding='utf-8') as csv_file:
        # CSVの3列目（インデックス2）の値を抽出し、ヘッダー行を除外
        return [row[2] for row in csv.reader(csv_file) if len(row) >= 3][1:]

# 指定されたディレクトリから画像ファイルのパスを見つける関数
def find_image_paths(img_dir, extracted_numbers):
    image_paths = {}
    for root, _, files in os.walk(img_dir):
        for file in tqdm.tqdm(files, desc="ファイル処理中"):
            # 画像ファイルの拡張子をチェック
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                file_name, _ = os.path.splitext(file)
                file_parts = file_name.split('-')
                file_number = file_parts[0]
                # 抽出された番号と一致する画像を見つける
                if file_number in extracted_numbers:
                    current_path = os.path.join(root, file)
                    # 同じ番号の画像が複数ある場合、番号が小さい方を選択
                    if file_number not in image_paths or (len(file_parts) > 1 and int(file_parts[1]) < int(image_paths[file_number].split('-')[1])):
                        image_paths[file_number] = current_path
    return list(image_paths.values())

# 画像をコピーする関数
def copy_images(image_paths, destination_folder):
    for image_path in tqdm.tqdm(image_paths, desc="画像コピー中"):
        shutil.copy(image_path, os.path.join(destination_folder, os.path.basename(image_path)))

# メイン関数
def main():
    # 各種パスを設定
    img_dir = "F:\眼位写真NEW"
    csv_dir = r"C:\Users\ykita\OneDrive\デスクトップ\test_images"
    copy_dir = r"C:\Users\ykita\OneDrive\デスクトップ\test_images"

    # "Disease_extracted_"で始まるCSVファイルを探す
    csv_files = glob.glob(os.path.join(csv_dir, "Disease_extracted_*.csv"))

    for extract_csv_path in csv_files:
        # 新しいフォルダ名を生成し、作成
        csv_basename = os.path.splitext(os.path.basename(extract_csv_path).replace("Disease_extracted_", ""))[0]
        new_folder_path = os.path.join(copy_dir, csv_basename)
        print(f"作成しようとしているフォルダのパス: {new_folder_path}")

        try:
            os.makedirs(new_folder_path, exist_ok=True)
            print(f"フォルダが正常に作成されました: {new_folder_path}")
        except Exception as e:
            print(f"フォルダの作成中にエラーが発生しました: {e}")
            continue

        if not os.path.exists(new_folder_path):
            print(f"エラー: フォルダ {new_folder_path} が作成されませんでした。")
            continue

        # CSVから情報を抽出し、該当する画像を見つけてコピー
        extracted_numbers = extract_csv_info(extract_csv_path)
        image_paths = find_image_paths(img_dir, extracted_numbers)
        copy_images(image_paths, new_folder_path)

        # 処理完了メッセージを表示
        print(f"コピーが完了しました。{len(image_paths)}個の画像が{new_folder_path}にコピーされました。")

# スクリプトが直接実行された場合にメイン関数を呼び出す
if __name__ == "__main__":
    main()
